(Thanks to K589K589 for image-data reading method.)


This is the 3rd time for me to join Competition. I'm still a beginner.
Let's see how much Score I can have with Meta data only, first.
Then, I'll try to predict with CNN using Photo-image data.

コンペ参加は今回で3回目。まだまだ初心者。
まずはMeta Dataでどの程度のScoreが出るか見てみたいと思う。
その後、写真データを使ってシンプルなCNNで予測をしてみよう。

**Public Score**

**[ LGBM using Meta-data only ]**

   (1st) **20.49737** : Normal LightGBM  
   (2nd) **20.49137** : Add new features
   
**[ TensorFlow CNN using Photo-image data ]**

   (3rd) **20.73773** : TensorFlow CNN  (Filter 8 > 16 > 32)  
   (4th) **20.74033** : TensorFlow CNN  (Filter 32 > 32 > pooling >64 > 64 >pooling)


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import KFold
import lightgbm as lgbm
from sklearn.metrics import mean_squared_error

In [ ]:
train = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
test = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')

In [ ]:
train.head(20)

Meta Data is just a supplemental explanation and hence very simple table, like Eyes - 1or0 (Both eyes are facing front or near-front) , Face - 1or0 (Decently clear face, facing front), or something like..  From a look at row11 and row15, I can see even if the data itself is exactly same (Eyes, Face, Near is 1 and the rest is 0), the Pawpularity is totally different (2 and 98)...

Meta Dataは、Photoの付属説明なので、Eyes（目がちゃんと前を向いているか 1or0）、Face（顔がくっきりと前を向いているか 1or0）といった非常にシンプルなテーブル。11行目と15行目を見てもわかるが、まったく同じデータ（Eyes, Face, Nearが1。それ以外は0）でも、Pawpularityは2と98というように全然違う結果となっている。

Any specific columns which is deeply related to "Pawpularity"??
Let's see Heatmap.

何か特にPawpularityと相関の深そうなColumnはあるのかな？ヒートマップを見てみよう。

In [ ]:
train_corr = train.corr()
plt.figure(figsize = (13,13))
sns.heatmap(train_corr, vmax=1, vmin=-1, center=0,annot=True)

Hmm, do not find any columns specifically corresponding to Pawpularity... 

う～ん、Pawpularityと相関関係にあるField特に見当たらないけどなー

In [ ]:
train["Pawpularity"].plot.hist(bins=50)

I can see a feature in Pawpularity histogram. Basically, near to normal distribution, but Perfect score(100) seems extreamly many.

ターゲットとなるPawpularityは、ヒストグラムにちょっと特徴あるな。正規分布に近い形をしているものの、100（満点）が異常に多い。

# (1) Normal LightGBM

Once, let's throw the training data as it is without any preprocessing.
(Dividing in 3portion, and take the average)

いったん、トレーニングデータは何も処理せずに、そのままLightGBMにかけてみる。
（3分割してその平均をとる）

In [ ]:
def training_exe (train):
    kf = KFold(n_splits = 3)
    models = []
    rmses =[]
    categories = ["Id"]
    
    train["Id"] = train["Id"].astype('category')
    X_train = train.drop(['Pawpularity'], axis=1)
    Y_train = train['Pawpularity']
    
    lgbm_params = {
        "objective":"regression",
        "random_seed":1234
    }
    
    for train_index, val_index in kf.split(X_train):
        XX_train = X_train.iloc[train_index]
        XX_valid = X_train.iloc[val_index]
        
        YY_train = Y_train.iloc[train_index]
        YY_valid = Y_train.iloc[val_index]
        
        lgbm_train = lgbm.Dataset(XX_train, YY_train, categorical_feature = categories)
        lgbm_eval = lgbm.Dataset(XX_valid, YY_valid, categorical_feature = categories, reference=lgbm_train)
        
        model_lgbm = lgbm.train(lgbm_params,
                           lgbm_train,
                           valid_sets = lgbm_eval,
                           num_boost_round = 200,
                           early_stopping_rounds =20,
                           verbose_eval = 10,
                           )
        y_pred = model_lgbm.predict(XX_valid, num_iteration = model_lgbm.best_iteration)
        
        tmp_rmse = np.sqrt(mean_squared_error(YY_valid, y_pred))
        print (tmp_rmse)
        models.append(model_lgbm)
        rmses.append(tmp_rmse)
        
    ave_rmse = sum(rmses)/len(rmses)    
    
    return models, ave_rmse

In [ ]:
def pred_exe (test):
    preds=[]
    test["Id"] = test["Id"].astype('category')

    for model in models:
        pred =model.predict(test)
        preds.append(pred)
    
    preds_array = np.array(preds)
    preds_mean = np.mean(preds_array, axis=0)
    
    return preds_mean

In [ ]:
models, ave_rmse = training_exe (train)
ave_rmse

At this point, the RMSE shows 20.713.... Let's try test data using this model.

とりあえずトレーニングデータ結果は20.5825・・・。これでテストデータをかけてみますか。

In [ ]:
test.head()

In [ ]:
pred = pred_exe(test)

sub = pd.DataFrame()
sub['Id']=test['Id']
#sub['Pawpularity'] = pred

#sub.to_csv('submission.csv',index=False)
sub.head()

After predicting using test data, tried to submit. The result of Public Score is **20.49737** 

テストデータをモデルにかけてサブミッションしてみる。Public Scoreの結果は、**20.49737**

# (2) Adding new feature

Based on the guess that LGBM cannot predict very well since the original table is too simple, try to add new features using the existing fealds which seem corresponding to each other.
The result is, slightly improved to **20.49137**.

今のTrainingテーブルはシンプルすぎてうまく予測できないと思うので、関係のありそうなフィールドを組み合わせてあらたな特徴量を作ってみる。
結果は、**20.49137**へとわずかに改善。

In [ ]:
def add_feature(df) :
    df["Attractive"] = df["Eyes"] + df["Face"] + df["Near"] + df["Subject Focus"]
    df["Humantic"]   = df["Human"] + df["Collage"]
    df["Addition"]   = df["Accessory"] + df["Info"]
    return df  

In [ ]:
train_df1 = train.copy()
add_feature(train_df1)

In [ ]:
models, ave_rmse = training_exe (train_df1)
ave_rmse

In [ ]:
test_df1 = test.copy()

add_feature(test)

In [ ]:
pred = pred_exe(test)

#sub = pd.DataFrame()
#sub['Id']=test['Id']
sub['Pawpularity'] = pred
#sub.to_csv('submission.csv',index=False)
sub.head()

# (3) Normal TensorFlow CNN using Photo-image data



Next, let's try to predict using CNN for actual Photo-image data.

では、実際の写真データをCNNで画像認識させて、予測してみよう。

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import KFold
import lightgbm as lgbm
from sklearn.metrics import mean_squared_error
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import cv2

In [ ]:
import missingno as msno

train=pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
test=pd.read_csv('../input/petfinder-pawpularity-score/test.csv')

In [ ]:
train["file_path"] = train["Id"].apply(lambda x: "../input/petfinder-pawpularity-score/train/" + x + ".jpg")
test["file_path"] = test["Id"].apply(lambda x: "../input/petfinder-pawpularity-score/test/" + x + ".jpg")

In [ ]:
%%time

def preprocess(image_url):
    image_string = tf.io.read_file(image_url)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.central_crop(image, 1.0)
    image = tf.image.resize(image, (128, 128))
    return image
x_train=[]
for i in train['file_path']:
    x1=preprocess(i)
    x_train.append(x1)

It takes a bit time to read Photo-image data..
Let's see some actual photos 

写真イメージの読み込みは少し時間かかるなぁー　実際の写真を少し見てみる。

In [ ]:
plt.figure(figsize=(20, 20))
row, col = 5, 4
for i in range(row * col):
    plt.subplot(row, col, i+1)
    image = cv2.imread(train.loc[i, 'file_path'])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    target = train.loc[i, 'Pawpularity']
    plt.imshow(image)
    plt.title(f"No: {i}" f"   Pawpularity: {target}")
plt.show()

Hmmm...Interesting. I guess Pawpularity means "Popularity", but why No.11 is pawpularity=2, which is almost the worst, while No.19 is pawpularity =100, which is perfect score??  I understand No.15 is pawpularity=98 (Cute!), but cannot understand why No.19 is perfect score... OK, let's ignore and go ahead.

ふむふむ。Pawpularityって、「人気度」（＝popularity）のはずなんだけど、No.11が人気度2とほぼ最低点で、No.19が人気度100と満点なんだ。No.15の人気度98はなんとなく分かるけど、No.19がなんで満点なの？？　まぁ、いいや。気にせず先に進みましょう。

In [ ]:
test1=[]
for i in test['file_path']:
    x1=preprocess(i)
    test1.append(x1)
test1=np.array(test1)

In [ ]:
x_train=np.array(x_train)
y_train=train['Pawpularity']
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_train,y_train,test_size=0.2)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers

In [ ]:
model = Sequential()
input_shape = (128, 128, 3)

In [ ]:

# Layer1 : CNN1
model.add(
    Conv2D(
        filters=32,
        kernel_size = (3,3),
        strides =2,
        padding = 'same',
        activation = 'relu',
        input_shape = input_shape,
    ))

# Layer2 : CNN2
model.add(
    Conv2D(
        filters=64,
        kernel_size = (3,3),
        strides =2,
        padding = 'same',
        activation = 'relu',        
    ))

# Layer3 : CNN3
model.add(
    Conv2D(
        filters=32,
        kernel_size = (3,3),
        strides =2,
        padding = 'same',
        activation = 'relu',        
    ))

# Flatten
model.add(Flatten())

# Layer4
model.add(
    Dense(128, activation = 'relu'))
model.add(Dropout(0.25))

# Output
model.add(Dense(1))

# Compile
model.compile(
    loss = tf.keras.losses.MeanSquaredError(),    
    metrics=[tf.keras.metrics.RootMeanSquaredError(name="rmse"), "mae", "mape"],
    optimizer = tf.keras.optimizers.Adam(1e-3)
)


In [ ]:
total_train = x_train.shape[0]
total_validate = y_train.shape[0]
batch_size = 64

In [ ]:

%%time

import math
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping, Callback

def step_decay(epoch):
    initial_lrate = 0.001
    drop = 0.5
    epochs_drop = 10.0
    lrate = initial_lrate * math.pow(
        drop,
        math.floor((epoch)/epochs_drop)
    )
    return lrate

lrate = LearningRateScheduler(step_decay)

earstop = EarlyStopping(
    monitor = 'val_loss',
    min_delta = 0,
    patience = 5)

history = model.fit(
    x_train,
    y_train,
    epochs = 20,
    batch_size=64,
    validation_data = (x_test,y_test),
    verbose = 1,
    callbacks = [lrate, earstop]
)


In [ ]:
cnn_pred=model.predict(test1)

In [ ]:
#sub=pd.DataFrame()
#sub['Id']=test['Id']
sub['Pawpularity']=cnn_pred
#sub.to_csv('submission.csv',index=False)
sub

After submitting, Public score shows **20.73773**... Much worse than simple LGBM using Meta-data only... Actual Leaderboard will evaluate the prediction using Photo-image data... Let's try to improve.

Submitしてみると、Public Scoreは**20.73773**。Meta-dataを使っただけのSimpleなLGBMより悪い。。。実際はこちらの写真データを使った予測が評価されるので、これから改善を図っていこう。。

# (4) TF CNN (change Filters and add Pooling layer)

As an attempt, changed filters and added Pooling layers.  
(Filter 32 > 32 > Pooling > 64 > 64 > Pooling)  
The score shows **20.74033**. Slightly got worse or almost unchanged..

試しに、Filter数を変えてプーリング層を追加してみる。  
(Filter 32 > 32 > Pooling > 64 > 64 > Pooling)  
が、スコアは**20.74033**と、若干悪化というかほぼ変わらず。。


One more try to change CNN (Filter 8 > 16 > Pooling > 32 > 64)  
However, the result of Public score is **20.83332**, which is further worse... Umm.. it may not be good to just change the CNN parameters...

さらに、変えてみる（Filter 8 > 16 > Pooling > 32 > 64）。  
が、結果は**20.83332**とさらに悪化。う～ん、CNNのパラメーター変えるだけだとダメなのかな。。。

Just as an attempt, try to simply ensemble the result of LGBM and CNN (may not make sense sinse the data source is different...)  
The result is, Public Score shows **20.60492**, which is just slightly better than the simple average of LGBM-only 20.49137 and CNN-only 20.73773... As assumed, does not work very well..hmm..

LGBMの結果とCNNの結果をアンサンブルしてみる（データソースが違うので余り意味はないかもしれないが。。。）  
結果は、Public Score　**20.60492**となった。LGBM単独スコア20.49137とCNN単独スコア20.73773の単純平均よりわずかに良い程度。やっぱりあまり意味なかったかなぁ。

In [ ]:
sub['Pawpularity'] = pred*0.5 + cnn_pred*0.5
sub.to_csv('submission.csv',index=False)
sub